# I. Import thư viện

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import islice
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,classification_report
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils.class_weight import compute_sample_weight
import pickle
from joblib import dump, load

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

# II. Đọc dữ liệu

In [5]:
#df = pd.read_csv('/content/drive/MyDrive/DS317/df.csv')
df = pd.read_csv('D:\Learning\Semester 1 year 4\DS317\Project\Data\chuaxuly.csv')
df = df.drop(columns=['Unnamed: 0'])

In [6]:
df.head(5)

,mssv,soquyetdinh,ngaycapvb,namsinh,gioitinh,noisinh,lopsh,khoa,hedt,khoahoc,...,drlnam6,drl_3_5,CCHV,dien_tt,diem_tt,lop12_matinh,lop12_matruong,tienganh,so_cc,label
0,BE375BAAXPvAibaEXe9JDlHA4z2GHJ3/PVStCxR2,204/QĐ-ĐHCNTT,12/04/2019,1995.0,1.0,TP. Hồ Chí Minh,KTPM0001,CNPM,CQUI,8.0,...,55.0,64.0,NaN,THPT,31.0,2.0,102.0,NaN,NaN,Khá
1,2420ED57XPvAibaEXe/Lh6v1XxTKJa6JLFRUPkLM,675/QĐ-ĐHCNTT,11/10/2019,1995.0,1.0,Đồng Tháp,HTTT0001,HTTT,CTTT,8.0,...,NaN,64.0,1.0,THPT,25.5,2.0,78.0,NaN,NaN,Rớt
2,83B76C01XPvAibaEXe/lOccskaOiO2K46r7t4qnt,NaN,NaN,1994.0,1.0,Hà Nam Ninh,KHMT2013,KHMT,CQUI,8.0,...,NaN,0.0,1.0,THPT,26.5,48.0,3.0,NaN,NaN,Rớt
3,91F785ABXPvAibaEXe/lOccskaOiO5y4GbVvuRQu,204/QĐ-ĐHCNTT,12/04/2019,1995.0,1.0,TP. Hồ Chí Minh,HTTT0001,HTTT,CTTT,8.0,...,55.0,62.0,NaN,THPT,24.5,2.0,4.0,TOEIC_LR + 'TOEIC_LR',2.0,Rớt
4,007C275DXPvAibaEXe+TFgEDwYNnveOOmOYeYzF6,NaN,NaN,1995.0,1.0,Thành phố Hồ Chí Minh,MMTT0001,MMT&TT,CQUI,8.0,...,NaN,66.0,2.0,THPT,31.0,2.0,57.0,NaN,NaN,Rớt


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7826 entries, 0 to 7825
Data columns (total 38 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   mssv            7826 non-null   object 
 1   soquyetdinh     1572 non-null   object 
 2   ngaycapvb       1572 non-null   object 
 3   namsinh         7826 non-null   float64
 4   gioitinh        7826 non-null   float64
 5   noisinh         7826 non-null   object 
 6   lopsh           7826 non-null   object 
 7   khoa            7826 non-null   object 
 8   hedt            7826 non-null   object 
 9   khoahoc         7826 non-null   float64
 10  chuyennganh2    7826 non-null   object 
 11  tinhtrang       7826 non-null   float64
 12  diachi_tinhtp   7826 non-null   object 
 13  diemnamhoc1     7798 non-null   float64
 14  diemnamhoc2     7407 non-null   float64
 15  diemnamhoc3     7087 non-null   float64
 16  diemnamhoc4     6788 non-null   float64
 17  diemnamhoc5     3322 non-null   f

In [6]:
df.isnull().sum()

mssv                 0
soquyetdinh       6253
ngaycapvb         6253
namsinh              0
gioitinh             0
noisinh              0
lopsh                0
khoa                 0
hedt                 0
khoahoc              0
chuyennganh2         0
tinhtrang            0
diachi_tinhtp        0
diemnamhoc1         28
diemnamhoc2        419
diemnamhoc3        739
diemnamhoc4       1038
diemnamhoc5       4503
diemnamhoc6       7038
diem_3_5          1071
dtb_toankhoa        25
dtb_tichluy         25
sotc_tichluy        25
mamh              3233
drlnam1             78
drlnam2            419
drlnam3            739
drlnam4           1038
drlnam5           4503
drlnam6           7038
drl_3_5           1009
dien_tt             27
diem_tt             27
lop12_matinh        27
lop12_matruong      27
sotc_rot1         3777
sotc_rot2         5349
sotc_rot3         5877
sotc_rot4         6709
sotc_rot5         7478
sotc_rot6         7722
tc_rot_tong       2857
rotmon_3_5        7412
tctichluy1 

In [8]:
df = df.drop(columns=['diemnamhoc4', 'diemnamhoc5', 'diemnamhoc6', 'drlnam4', 'drlnam5', 'drlnam6', 'dtb_toankhoa',
       'dtb_tichluy', 'sotc_tichluy'])

In [9]:
df['label'].value_counts()

Rớt                3038
Chưa tốt nghiệp    2873
Khá                1122
Giỏi                460
Trung bình khá      209
Trung bình          111
Xuất sắc             13
Name: label, dtype: int64

In [10]:
df = df.rename(columns={'label': 'xeploai'})

In [11]:
df_1 = df.copy()
df_2 = df.copy()

In [12]:
xl_dict = {
    'Rớt': 0,
    'Chưa tốt nghiệp': 0,
    'Trung bình': 1,
    'Trung bình khá': 2,
    'Khá': 3,
    'Giỏi': 4,
    'Xuất sắc': 5
}

In [13]:
def encode_xl(df):
  return xl_dict[df['xeploai']]

df_1['xeploai'] = df_1.apply(encode_xl, axis=1)

In [14]:
df_2 = df_2[df_2['xeploai'] != 'Chưa tốt nghiệp']

In [15]:
cate_col = df.select_dtypes(include=['object'])
num_col = df.select_dtypes(include=['int64', 'float64'])

In [16]:
for u in num_col:
    df_1[u] = df_1[u].fillna(0)
    df_2[u] = df_2[u].fillna(0)
for u in cate_col:
    df_1[u] = df_1[u].fillna('None')
    df_2[u] = df_2[u].fillna('None')
    
encode = preprocessing.LabelEncoder()
for u in cate_col:
    df_1[u] = encode.fit_transform(df_1[u])
    df_2[u] = encode.fit_transform(df_2[u])

In [17]:
df_1.head(5)

,mssv,soquyetdinh,ngaycapvb,namsinh,gioitinh,noisinh,lopsh,khoa,hedt,khoahoc,...,drlnam3,drl_3_5,CCHV,dien_tt,diem_tt,lop12_matinh,lop12_matruong,tienganh,so_cc,xeploai
0,5733,3,20,1995.0,1.0,204,81,0,2,8.0,...,85.0,64.0,0.0,4,31.0,2.0,102.0,9,0.0,3
1,1057,29,18,1995.0,1.0,225,42,1,3,8.0,...,82.0,64.0,1.0,4,25.5,2.0,78.0,9,0.0,0
2,3976,41,43,1994.0,1.0,176,62,2,2,8.0,...,-14.0,0.0,1.0,4,26.5,48.0,3.0,9,0.0,0
3,4404,3,20,1995.0,1.0,204,42,1,3,8.0,...,66.5,62.0,0.0,4,24.5,2.0,4.0,10,2.0,0
4,18,41,43,1995.0,1.0,208,94,5,2,8.0,...,78.5,66.0,2.0,4,31.0,2.0,57.0,9,0.0,0


In [18]:
df_1.to_csv('D:\Learning\Semester 1 year 4\DS317\Project\Data\Raw_Data_IncludeCTN_For_Models.csv', index=False)

In [19]:
df_2.to_csv('D:\Learning\Semester 1 year 4\DS317\Project\Data\Raw_Data_ExcludeCTN_For_Models.csv', index=False)